In [1]:
import wandb; wandb.login()
from transformers import GPT2TokenizerFast, GPTNeoForCausalLM, GPTNeoConfig
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
from datasets import load_dataset, load_from_disk
from transformers import RobertaForCausalLM

from tokenizers import Tokenizer, pre_tokenizers, decoders, AddedToken
from tokenizers.models import BPE, Unigram, WordLevel, WordPiece
from tokenizers.trainers import BpeTrainer, WordLevelTrainer, \
                                WordPieceTrainer, UnigramTrainer

from tokenizers.processors import RobertaProcessing

from tokenizers.pre_tokenizers import Whitespace

wandb: Currently logged in as: r-bragamedeirosmotaborges (tiny-transformers). Use `wandb login --relogin` to force relogin


In [2]:
def get_texts(dataset, split='train'):
    for example in dataset[split]:
        yield example['text']

In [15]:
dataset = load_dataset('openwebtext', num_proc=16)

/home/prometheus/TUDelft/cse/year3/RP/tiny-transformers/code/rafael-tokenisation/.venv/lib/python3.10/site-packages/datasets/load.py:1486: FutureWarning: The repository for openwebtext contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/openwebtext
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Generating train split:   0%|          | 0/8013769 [00:00<?, ? examples/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Loading dataset shards:   0%|          | 0/83 [00:00<?, ?it/s]

In [ ]:
tokenizer = Tokenizer(BPE(
    dropout=None,
    unk_token=None,
    continuing_subword_prefix="Ġ",
    end_of_word_suffix="",
    fuse_unk=False,
    byte_fallback=False,
    ignore_merges=False
))

tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False, trim_offsets=True, use_regex=True)
tokenizer.decoder = decoders.ByteLevel(add_prefix_space=True, trim_offsets=True, use_regex=True)

# Configure post-processor
tokenizer.post_processor = RobertaProcessing(
    sep=("</s>", 2),
    cls=("<s>", 1),
    trim_offsets=True,
    add_prefix_space=False
)

# Add special tokens
special_tokens = [
    AddedToken("<pad>", single_word=False, lstrip=False, rstrip=False, normalized=True, special=True),
    AddedToken("<s>", single_word=False, lstrip=False, rstrip=False, normalized=True, special=True),
    AddedToken("</s>", single_word=False, lstrip=False, rstrip=False, normalized=True, special=True),
    AddedToken("<unk>", single_word=False, lstrip=False, rstrip=False, normalized=True, special=True),
]

# Create the trainer
trainer = BpeTrainer(
    vocab_size=10000, 
    special_tokens=[str(t.content) for t in special_tokens]
)

# Train the tokenizer
tokenizer.train_from_iterator(get_texts(dataset, 'train'), trainer=trainer)

# Manually add the special tokens with detailed configurations
tokenizer.add_special_tokens([t.content for t in special_tokens])


In [13]:
tokenizer.save("test-tokenizers/10k-bpe-tinystories/tokenizer.json")
tokenizer.model.save("test-tokenizers/10k-bpe-tinystories")

['test-tokenizers/10k-bpe-tinystories/vocab.json',
 'test-tokenizers/10k-bpe-tinystories/merges.txt']

In [10]:
tokenizer = GPT2TokenizerFast.from_pretrained(
        'test-tokenizers/10k-bpe-tinystories',                                         # our custom tokenizer
        model_max_length=512    # sequence length (context window)
    )

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RobertaTokenizer'. 
The class this function is called from is 'GPT2TokenizerFast'.
